Movie Studio Analysis

Overview

Business Problem

Your company now sees all the big companies creating original video content and they want to get in on the fun. They have decided to create a new movie studio, but they don’t know anything about creating movies. You are charged with exploring what types of films are currently doing the best at the box office. You must then translate those findings into actionable insights that the head of your company's new movie studio can use to help decide what type of films to create.

In the last 10 years, which genre was the most profitable at the box office. profits between genres. 

Most popular english movies filtered by user rating???
filters - year, english, gross profit, user rating, best director
3 recommendations?
comedy vs drama


** to make 2x to 3x more profit 
set budget = movies under between 5-80 million, medium budget movies
genre of movies - filter

Data Understanding

In [33]:
import pandas as pd
import numpy as np 
from scipy import stats

import sqlite3
import zipfile 

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [23]:
# Read in Movie gross dataset with formatting and coding specifications
movie_gross = pd.read_csv('data/bom.movie_gross.csv.gz')
movie_gross.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [5]:
movie_gross.head()

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [6]:
movie_gross.describe()

,domestic_gross,year
count,3.359000e+03,3387.000000
mean,2.874585e+07,2013.958075
std,6.698250e+07,2.478141
min,1.000000e+02,2010.000000
25%,1.200000e+05,2012.000000
50%,1.400000e+06,2014.000000
75%,2.790000e+07,2016.000000
max,9.367000e+08,2018.000000


In [24]:
#checking the year column to see how far the data goes
movie_gross['year'].value_counts().sort_index(ascending=True)

year
2010    328
2011    399
2012    400
2013    350
2014    395
2015    450
2016    436
2017    321
2018    308
Name: count, dtype: int64

In [22]:
#Read in Rotten Tomatoes Movie info dataset
movie_info = pd.read_csv('Data/rt.movie_info.tsv.gz', sep='\t', encoding='latin')
movie_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
 7   dvd_date      1201 non-null   object
 8   currency      340 non-null    object
 9   box_office    340 non-null    object
 10  runtime       1530 non-null   object
 11  studio        494 non-null    object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB


In [9]:
movie_info.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [10]:
movie_info['genre'].value_counts()

genre
Drama                                                                                     151
Comedy                                                                                    110
Comedy|Drama                                                                               80
Drama|Mystery and Suspense                                                                 67
Art House and International|Drama                                                          62
                                                                                         ... 
Art House and International|Drama|Sports and Fitness                                        1
Comedy|Documentary|Musical and Performing Arts|Special Interest                             1
Comedy|Cult Movies|Mystery and Suspense|Science Fiction and Fantasy                         1
Action and Adventure|Art House and International|Mystery and Suspense|Special Interest      1
Comedy|Drama|Kids and Family|Sports and Fitness       

In [21]:
movie_info['director'].value_counts()

director
Steven Spielberg      10
Clint Eastwood         8
William Friedkin       4
Curtis Hanson          4
William Beaudine       4
                      ..
Evans Butterworth      1
Jeannot Szwarc         1
Maroun Bagdadi         1
James Hogan            1
David Mickey Evans     1
Name: count, Length: 1125, dtype: int64

In [25]:
# Read in Rotten Tomatoes reviews data
rt_reviews = pd.read_csv('data/rt.reviews.tsv.gz', sep='\t', encoding='latin')
rt_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54432 entries, 0 to 54431
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          54432 non-null  int64 
 1   review      48869 non-null  object
 2   rating      40915 non-null  object
 3   fresh       54432 non-null  object
 4   critic      51710 non-null  object
 5   top_critic  54432 non-null  int64 
 6   publisher   54123 non-null  object
 7   date        54432 non-null  object
dtypes: int64(2), object(6)
memory usage: 3.3+ MB


In [26]:
rt_reviews.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [27]:
# Read in tmdb movies dataset
tmdb_movies = pd.read_csv('data/tmdb.movies.csv.gz')
tmdb_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         26517 non-null  int64  
 1   genre_ids          26517 non-null  object 
 2   id                 26517 non-null  int64  
 3   original_language  26517 non-null  object 
 4   original_title     26517 non-null  object 
 5   popularity         26517 non-null  float64
 6   release_date       26517 non-null  object 
 7   title              26517 non-null  object 
 8   vote_average       26517 non-null  float64
 9   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(3), object(5)
memory usage: 2.0+ MB


In [28]:
tmdb_movies.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [20]:
tmdb_movies['original_language'] == 'en'
language_counts = tmdb_movies['original_language'].value_counts()
language_counts


original_language
en    23291
fr      507
es      455
ru      298
ja      265
      ...  
bo        1
si        1
sl        1
hz        1
dz        1
Name: count, Length: 76, dtype: int64

In [29]:
# Read in movie budgets dataset
movie_budgets = pd.read_csv('data/tn.movie_budgets.csv.gz')
movie_budgets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [30]:
movie_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [31]:
#Filter data by release date last 10 years, create new column year
movie_budgets['year'] = movie_budgets['release_date'].str[-4:].astype(int)
movie_budgets


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,year
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279",2009
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875",2011
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350",2019
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963",2015
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747",2017
...,...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0,2018
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495",1999
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338",2005
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0,2015


In [35]:
# Connect and Read in SQL tables
conn = sqlite3.connect('im.db.zip')
zip_file_path = 'data/im.db.zip'

with zipfile.ZipFile('data/im.db.zip', 'r') as zip_ref:
    zip_ref.extractall('unzippedData')

database_path = 'unzippedData/im.db'
conn = sqlite3.connect(database_path)

In [36]:
schema_df = pd.read_sql("""

SELECT *
FROM sqlite_master

""", conn)

schema_df


,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


In [60]:
# reading the movie_basics table
movie_basics = pd.read_sql(
    """
    SELECT *
    FROM movie_basics
    """, conn
)
movie_basics

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"
...,...,...,...,...,...,...
146139,tt9916538,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123.0,Drama
146140,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,NaN,Documentary
146141,tt9916706,Dankyavar Danka,Dankyavar Danka,2013,NaN,Comedy
146142,tt9916730,6 Gunn,6 Gunn,2017,116.0,None


In [44]:
# distinct movie genres
pd.read_sql(
    """
    SELECT DISTINCT genres
    FROM movie_basics
    """, conn
)

,genres
0,"Action,Crime,Drama"
1,"Biography,Drama"
2,Drama
3,"Comedy,Drama"
4,"Comedy,Drama,Fantasy"
...,...
1081,"History,Reality-TV,War"
1082,"Mystery,Reality-TV,Thriller"
1083,"Music,Musical,Reality-TV"
1084,"Animation,Crime"


In [46]:
# reading the movie ratings table
pd.read_sql(
    """
    SELECT *
    FROM movie_ratings
    """, conn
)


,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21
...,...,...,...
73851,tt9805820,8.1,25
73852,tt9844256,7.5,24
73853,tt9851050,4.7,14
73854,tt9886934,7.0,5


In [47]:
#reading in the movie_akas table
pd.read_sql(
    """
    SELECT *
    FROM movie_akas
    """, conn
)

,movie_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0369610,10,Джурасик свят,BG,bg,None,None,0.0
1,tt0369610,11,Jurashikku warudo,JP,None,imdbDisplay,None,0.0
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,None,imdbDisplay,None,0.0
3,tt0369610,13,O Mundo dos Dinossauros,BR,None,None,short title,0.0
4,tt0369610,14,Jurassic World,FR,None,imdbDisplay,None,0.0
...,...,...,...,...,...,...,...,...
331698,tt9827784,2,Sayonara kuchibiru,None,None,original,None,1.0
331699,tt9827784,3,Farewell Song,XWW,en,imdbDisplay,None,0.0
331700,tt9880178,1,La atención,None,None,original,None,1.0
331701,tt9880178,2,La atención,ES,None,None,None,0.0


In [48]:
#joining movie_basics table and movie_ratings table 
pd.read_sql(
    """
    SELECT DISTINCT b.primary_title, b.start_year, b.genres, r.averagerating
    FROM movie_basics b
    JOIN movie_ratings r
    ON b.movie_id = r.movie_id
    """, conn
)

,primary_title,start_year,genres,averagerating
0,Sunghursh,2013,"Action,Crime,Drama",7.0
1,One Day Before the Rainy Season,2019,"Biography,Drama",7.2
2,The Other Side of the Wind,2018,Drama,6.9
3,Sabse Bada Sukh,2018,"Comedy,Drama",6.1
4,The Wandering Soap Opera,2017,"Comedy,Drama,Fantasy",6.5
...,...,...,...,...
73849,Diabolik sono io,2019,Documentary,6.2
73850,Sokagin Çocuklari,2019,"Drama,Family",8.7
73851,Albatross,2017,Documentary,8.5
73852,La vida sense la Sara Amat,2019,None,6.6


In [49]:
#joining movie_basics and movie_akas table
pd.read_sql(
    """
    SELECT b.primary_title, b.start_year, b.genres, a.region, a.language
    FROM movie_basics b
    JOIN movie_akas a
    ON b.movie_id = a.movie_id
    LIMIT 50
    """, conn
)

,primary_title,start_year,genres,region,language
0,Sunghursh,2013,"Action,Crime,Drama",None,None
1,Sunghursh,2013,"Action,Crime,Drama",IN,None
2,Sunghursh,2013,"Action,Crime,Drama",IN,hi
3,Sunghursh,2013,"Action,Crime,Drama",IN,hi
4,Sunghursh,2013,"Action,Crime,Drama",IN,hi
5,One Day Before the Rainy Season,2019,"Biography,Drama",None,None
6,One Day Before the Rainy Season,2019,"Biography,Drama",None,None
7,One Day Before the Rainy Season,2019,"Biography,Drama",IN,None
8,One Day Before the Rainy Season,2019,"Biography,Drama",XWW,en
9,The Other Side of the Wind,2018,Drama,None,None


In [52]:
#sorting by start_year
pd.read_sql(
    """
    SELECT DISTINCT start_year
    FROM movie_basics
    ORDER BY start_year Desc
    """, conn
)

,start_year
0,2115
1,2027
2,2026
3,2025
4,2024
5,2023
6,2022
7,2021
8,2020
9,2019


In [55]:
# joining 2 csv files together

merged_df = pd.merge(tmdb_movies, movie_budgets, left_on='title', right_on='movie', how='inner')
merged_df

,Unnamed: 0,genre_ids,id_x,original_language,original_title,popularity,release_date_x,title,vote_average,vote_count,id_y,release_date_y,movie,production_budget,domestic_gross,worldwide_gross,year
0,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610,30,"Mar 26, 2010",How to Train Your Dragon,"$165,000,000","$217,581,232","$494,870,992",2010
1,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368,15,"May 7, 2010",Iron Man 2,"$170,000,000","$312,433,331","$621,156,389",2010
2,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174,37,"Nov 22, 1995",Toy Story,"$30,000,000","$191,796,233","$364,545,516",1995
3,2473,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174,37,"Nov 22, 1995",Toy Story,"$30,000,000","$191,796,233","$364,545,516",1995
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186,38,"Jul 16, 2010",Inception,"$160,000,000","$292,576,195","$835,524,642",2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2380,26323,[],509316,en,The Box,0.600,2018-03-04,The Box,8.0,1,66,"Nov 6, 2009",The Box,"$25,000,000","$15,051,977","$34,356,760",2009
2381,26425,[10402],509306,en,The Box,0.600,2018-03-04,The Box,6.0,1,66,"Nov 6, 2009",The Box,"$25,000,000","$15,051,977","$34,356,760",2009
2382,26092,"[35, 16]",546674,en,Enough,0.719,2018-03-22,Enough,8.7,3,68,"May 24, 2002",Enough,"$38,000,000","$39,177,215","$50,970,660",2002
2383,26322,[],513161,en,Undiscovered,0.600,2018-04-07,Undiscovered,8.0,1,7,"Aug 26, 2005",Undiscovered,"$9,000,000","$1,069,318","$1,069,318",2005


In [62]:
#joining to sql table 
final_merged_df = pd.merge(merged_df, movie_basics, on='original_title', how='inner')
final_merged_df 

,Unnamed: 0,genre_ids,id_x,original_language,original_title,popularity,release_date_x,title,vote_average,vote_count,...,movie,production_budget,domestic_gross,worldwide_gross,year,movie_id,primary_title,start_year,runtime_minutes,genres
0,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610,...,How to Train Your Dragon,"$165,000,000","$217,581,232","$494,870,992",2010,tt0892769,How to Train Your Dragon,2010,98.0,"Action,Adventure,Animation"
1,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368,...,Iron Man 2,"$170,000,000","$312,433,331","$621,156,389",2010,tt1228705,Iron Man 2,2010,124.0,"Action,Adventure,Sci-Fi"
2,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186,...,Inception,"$160,000,000","$292,576,195","$835,524,642",2010,tt1375666,Inception,2010,148.0,"Action,Adventure,Sci-Fi"
3,5,"[12, 14, 10751]",32657,en,Percy Jackson & the Olympians: The Lightning T...,26.691,2010-02-11,Percy Jackson & the Olympians: The Lightning T...,6.1,4229,...,Percy Jackson & the Olympians: The Lightning T...,"$95,000,000","$88,768,303","$223,050,874",2010,tt0814255,Percy Jackson & the Olympians: The Lightning T...,2010,118.0,"Adventure,Family,Fantasy"
4,7,"[16, 10751, 35]",10193,en,Toy Story 3,24.445,2010-06-17,Toy Story 3,7.7,8340,...,Toy Story 3,"$200,000,000","$415,004,880","$1,068,879,522",2010,tt0435761,Toy Story 3,2010,103.0,"Adventure,Animation,Comedy"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3937,26425,[10402],509306,en,The Box,0.600,2018-03-04,The Box,6.0,1,...,The Box,"$25,000,000","$15,051,977","$34,356,760",2009,tt5047858,The Box,2015,100.0,"Drama,War"
3938,26425,[10402],509306,en,The Box,0.600,2018-03-04,The Box,6.0,1,...,The Box,"$25,000,000","$15,051,977","$34,356,760",2009,tt5633056,The Box,2014,NaN,None
3939,26425,[10402],509306,en,The Box,0.600,2018-03-04,The Box,6.0,1,...,The Box,"$25,000,000","$15,051,977","$34,356,760",2009,tt5678896,The Box,2018,56.0,Thriller
3940,26092,"[35, 16]",546674,en,Enough,0.719,2018-03-22,Enough,8.7,3,...,Enough,"$38,000,000","$39,177,215","$50,970,660",2002,tt2926198,Enough,2013,96.0,"Drama,History"
